In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [2]:
  #accessing the html page and storing it into a BS object
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

  #creating a list to append values
raw_data=[]
  #targeting the table within the page
table = soup.find('table', class_='wikitable sortable')

  #iterating through each row in the table and appending the fields into the list
for word in table.find_all('td'):
    value = word.text.replace('\n','')
    raw_data.append(value)
  
   #reshaping the list to a to 288 rows x 3 columns. Then creating a dataframe on the reshaped list
df_postcode = pd.DataFrame(np.reshape(raw_data,(288,3)),columns=['Postal Code','Borough','Neighbourhood'])

    #deleting the rows, by index, of those records that contain a Borough value of Not assigned. 
df_postcode.drop((df_postcode[df_postcode['Borough']=='Not assigned'].index), inplace=True) 

In [3]:
combined=[] 
    #casting the post codes to a set() to remove duplicates, then casting back into a list to iterate using the for loop below
unique_postcodes = list(set(df_postcode['Postal Code']))

  #iterating through the dataframe to combine the neighbourhoods based on unique postcode. This also will check for any not assigned neighbourhoods and assign them the borough value
for i in range(len(unique_postcodes)):
    #returns a dataframe containing only the records of the current postcode, [i], in the list of unique_postcodes
      groupedby_postcode = df_postcode[df_postcode['Postal Code'] == unique_postcodes[i]]
  
    #checking for any "not assigned" neighbourhoods, and then hardcoding them to the borough name
      if groupedby_postcode['Neighbourhood'].any() == 'Not assigned':
        groupedby_postcode['Neighbourhood'] = groupedby_postcode['Borough']
    
    #setting delimited for the join function of the neighbourhoods
      temp_combine_neighbourhoods=', '
    #adding fields together into a new list:
      #first parameter = the current postcode string value
      #second parameter = the current borough value, by string
      #third parameter = the current neighbourhood values, joined by the delimiter above
      combined.append([unique_postcodes[i],list(set(groupedby_postcode['Borough']))[0],temp_combine_neighbourhoods.join(list(set(groupedby_postcode['Neighbourhood'])))])

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
   #reshaping the list to a to 103 rows x 3 columns. Then creating a dataframe on the reshaped list
df_combined = pd.DataFrame(np.reshape(combined,(103,3)),columns=['Postal Code','Borough','Neighborhood'])
print(np.shape(df_combined))
df_combined.head(12)

(103, 3)


,Postal Code,Borough,Neighborhood
0,M1E,Scarborough,"Morningside, Guildwood, West Hill"
1,M4H,East York,Thorncliffe Park
2,M2L,North York,"York Mills, Silver Hills"
3,M8Y,Etobicoke,"Royal York South East, Kingsway Park South Eas..."
4,M4A,North York,Victoria Village
5,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
6,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
7,M8X,Etobicoke,"Montgomery Road, The Kingsway, Old Mill North"
8,M5L,Downtown Toronto,"Victoria Hotel, Commerce Court"
9,M3C,North York,"Don Mills South, Flemingdon Park"


In [22]:
    #reading in the latitude and longitude coordinates from the csv file
df_lat_long=pd.read_csv("https://cocl.us/Geospatial_data")
    #merging the two dataframes together based on postal code
df_merged = pd.merge(df_combined, df_lat_long, on='Postal Code')
    #displaying a few lines of the merged table
df_merged.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711
1,M4H,East York,Thorncliffe Park,43.705369,-79.349372
2,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714
3,M8Y,Etobicoke,"Royal York South East, Kingsway Park South Eas...",43.636258,-79.498509
4,M4A,North York,Victoria Village,43.725882,-79.315572
5,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
6,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
7,M8X,Etobicoke,"Montgomery Road, The Kingsway, Old Mill North",43.653654,-79.506944
8,M5L,Downtown Toronto,"Victoria Hotel, Commerce Court",43.648198,-79.379817
9,M3C,North York,"Don Mills South, Flemingdon Park",43.725900,-79.340923



----------------------------------------------------------------------------------------------------------
For the third and final part of the assignment, i will focus just on the downtown toronto borough. 



In [23]:
    #getting the lat/long coordinates of toronto, using the geolocater module
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [25]:
    #create a new dataframe that focuses just on the downtown neighborhoods
dtowntoronto_data = df_merged[df_merged['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
dtowntoronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5L,Downtown Toronto,"Victoria Hotel, Commerce Court",43.648198,-79.379817
1,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M6G,Downtown Toronto,Christie,43.669542,-79.422564
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [27]:
    #create map of Toronto using latitude and longitude values
map_dtowntoronto = folium.Map(location=[latitude, longitude], zoom_start=11)

    #add markers to map
for lat, lng, label in zip(dtowntoronto_data['Latitude'], dtowntoronto_data['Longitude'], dtowntoronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dtowntoronto)  

    #display map
map_dtowntoronto

In [28]:
# @hidden_cell
#defining foursquare credentials
CLIENT_ID = 'RNBGL5BEPF5JT2ZT0KK4KHBEFTH0LRVTDAC5KQNCCTQZTBZU' # your Foursquare ID
CLIENT_SECRET = '41IV3LTCYNOFHJMTYFAPZC0UI3AYLWVYP41IMXQDF0NMGNVQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [30]:
    #reviewing lat/long of the first item in the new dataframe
neighborhood_latitude = dtowntoronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dtowntoronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = dtowntoronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Victoria Hotel, Commerce Court are 43.6481985, -79.37981690000001.


In [63]:
    #compiling the URL that we'll pass to the foursquare API 
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)




In [62]:
    #calling the foursquare API with the compiled URL. It returns a .json file with all the data, so storing that .json into a variable
results = requests.get(url).json()


In [33]:
    # function that extracts the category of the venue from the returned .json file from above
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [34]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

    #grab venue items
venues = results['response']['groups'][0]['items']
    
    #normalize the json so it can be read into a dataframe
nearby_venues = json_normalize(venues) # flatten JSON

    #setting filter columns as to what to bring into the dataframe (remove all the extra data not needed)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

    #apply filters to the entire line, and reassign to itself - i.e. filter out the fields not needed.
nearby_venues =nearby_venues.loc[:, filtered_columns]

    # filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

    # clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Canoe,Restaurant,43.647452,-79.381320
1,Equinox Bay Street,Gym,43.648100,-79.379989
2,Mos Mos Coffee,Café,43.648159,-79.378745
3,Walrus Pub & Beer Hall,Pub,43.647375,-79.379515
4,Pilot Coffee Roasters,Coffee Shop,43.648835,-79.380936


In [35]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # iterate through each returned result and apply only the filtered information
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        #create a new dataframe from the list built above
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
        #return the dataframe
    return(nearby_venues)

In [37]:
    #get a list of venues by calling the getNearbyVenues function and passing it the dtowntoronto dataframe and its coordinates.
toronto_venues = getNearbyVenues(names=dtowntoronto_data['Neighborhood'],
                                   latitudes=dtowntoronto_data['Latitude'],
                                   longitudes=dtowntoronto_data['Longitude']
                                  )

Victoria Hotel, Commerce Court
Berczy Park
Church and Wellesley
Christie
Central Bay Street
Rosedale
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
South Niagara, Railway Lands, Island airport, CN Tower, Harbourfront West, King and Spadina, Bathurst Quay
Richmond, Adelaide, King
Garden District, Ryerson
St. James Town, Cabbagetown
Regent Park, Harbourfront
Harbord, University of Toronto
St. James Town
Kensington Market, Grange Park, Chinatown


In [38]:
    #display the venues
print(toronto_venues.shape)
toronto_venues.head()

(1279, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Victoria Hotel, Commerce Court",43.648198,-79.379817,Canoe,43.647452,-79.381320,Restaurant
1,"Victoria Hotel, Commerce Court",43.648198,-79.379817,Equinox Bay Street,43.648100,-79.379989,Gym
2,"Victoria Hotel, Commerce Court",43.648198,-79.379817,Mos Mos Coffee,43.648159,-79.378745,Café
3,"Victoria Hotel, Commerce Court",43.648198,-79.379817,Walrus Pub & Beer Hall,43.647375,-79.379515,Pub
4,"Victoria Hotel, Commerce Court",43.648198,-79.379817,Pilot Coffee Roasters,43.648835,-79.380936,Coffee Shop


In [39]:
    #review the number of venues by neighborhood.
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
Central Bay Street,79,79,79,79,79,79
Christie,17,17,17,17,17,17
Church and Wellesley,85,85,85,85,85,85
"Design Exchange, Toronto Dominion Centre",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbord, University of Toronto",36,36,36,36,36,36
"Harbourfront East, Toronto Islands, Union Station",100,100,100,100,100,100


In [40]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 203 uniques categories.


In [41]:
# one hot encoding the venues by the type of the venue (venue category). This begins the process of the k-means clustering
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
toronto_onehot.shape

(1279, 203)

In [43]:
 #grouping the one-hot table by neighborhood. Then taking the mean calculation based on the "frequency" (since thats what the one-hot encoding was for) to determine order of appearance.
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.017544,0.000000,0.000000,0.000000,0.000000
1,Central Bay Street,0.012658,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.012658,...,0.000000,0.000000,0.00,0.00,0.00,0.012658,0.000000,0.000000,0.012658,0.000000
2,Christie,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Church and Wellesley,0.011765,0.011765,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.011765,...,0.011765,0.011765,0.00,0.00,0.00,0.000000,0.011765,0.011765,0.000000,0.011765
4,"Design Exchange, Toronto Dominion Centre",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,...,0.010000,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000
5,"First Canadian Place, Underground city",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.030000,...,0.010000,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000
6,"Garden District, Ryerson",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.010000,...,0.020000,0.000000,0.01,0.00,0.00,0.010000,0.010000,0.010000,0.010000,0.000000
7,"Harbord, University of Toronto",0.027778,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.027778,0.000000,0.00,0.00,0.00,0.000000,0.027778,0.000000,0.000000,0.000000
8,"Harbourfront East, Toronto Islands, Union Station",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.010000,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000
9,"Kensington Market, Grange Park, Chinatown",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.032258,0.000000,0.053763,0.010753,0.000000


In [44]:
toronto_grouped.shape

(18, 203)

In [45]:
    #setting the top x list
num_top_venues = 5

    #iterating through each neighborhood, then transposing the row into a  column of venue (as the row) by frequency (as the column header). 
    #Then resorting the largest frequency to the top and finally printing the neighborhoods top x list. Iterate through entire table.
for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.04
2  Farmers Market  0.04
3        Beer Bar  0.04
4          Bakery  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.14
1      Ice Cream Shop  0.05
2  Italian Restaurant  0.05
3                Café  0.04
4        Burger Joint  0.04


----Christie----
               venue  freq
0               Café  0.18
1      Grocery Store  0.18
2               Park  0.12
3  Convenience Store  0.06
4              Diner  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.08
1  Japanese Restaurant  0.06
2     Sushi Restaurant  0.05
3              Gay Bar  0.04
4           Restaurant  0.04


----Design Exchange, Toronto Dominion Centre----
                 venue  freq
0          Coffee Shop  0.15
1                 Café  0.08
2                Hotel  0.06
3  American Restaurant  0.04
4           Restaurant  0.04


----First Canadian Place, 

In [46]:
    #define a function to return a row containing the top x number of venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
    #defining the top x variable
num_top_venues = 10

    #numeric suffixes
indicators = ['st', 'nd', 'rd']

    # create columns according to number of top venues 
columns = ['Neighborhood']

    #create the columns of top x based on number defined above
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

    # create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

    #iterate through each top x column, and call the return_most_common_venues function to get the value for that neighborhood
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Cocktail Bar,Beer Bar,Farmers Market,Bakery,Steakhouse,Cheese Shop,Seafood Restaurant,Nightclub
1,Central Bay Street,Coffee Shop,Ice Cream Shop,Italian Restaurant,Sandwich Place,Burger Joint,Café,Spa,Japanese Restaurant,Salad Place,Bakery
2,Christie,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Diner,Convenience Store,Nightclub,Candy Store,Restaurant
3,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Gym,Hotel,Men's Store,Bubble Tea Shop,Mediterranean Restaurant
4,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Bar,Gastropub,Seafood Restaurant,Italian Restaurant,Deli / Bodega


In [48]:
from sklearn.cluster import KMeans
    # set number of clusters
kclusters = 5

 
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

    # run k-means clustering on the averaged one-hot encoded dataframe above
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 0, 0, 4, 0, 0], dtype=int32)

In [49]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dtowntoronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5L,Downtown Toronto,"Victoria Hotel, Commerce Court",43.648198,-79.379817,0,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Gym,Seafood Restaurant,Bakery,Steakhouse,Italian Restaurant
1,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Café,Cocktail Bar,Beer Bar,Farmers Market,Bakery,Steakhouse,Cheese Shop,Seafood Restaurant,Nightclub
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Gym,Hotel,Men's Store,Bubble Tea Shop,Mediterranean Restaurant
3,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Diner,Convenience Store,Nightclub,Candy Store,Restaurant
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Ice Cream Shop,Italian Restaurant,Sandwich Place,Burger Joint,Café,Spa,Japanese Restaurant,Salad Place,Bakery


In [50]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Victoria Hotel, Commerce Court",0,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Gym,Seafood Restaurant,Bakery,Steakhouse,Italian Restaurant
1,Berczy Park,0,Coffee Shop,Café,Cocktail Bar,Beer Bar,Farmers Market,Bakery,Steakhouse,Cheese Shop,Seafood Restaurant,Nightclub
2,Church and Wellesley,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Gym,Hotel,Men's Store,Bubble Tea Shop,Mediterranean Restaurant
4,Central Bay Street,0,Coffee Shop,Ice Cream Shop,Italian Restaurant,Sandwich Place,Burger Joint,Café,Spa,Japanese Restaurant,Salad Place,Bakery
6,"Harbourfront East, Toronto Islands, Union Station",0,Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Restaurant,Italian Restaurant,Scenic Lookout,Brewery,Sporting Goods Shop
7,"Design Exchange, Toronto Dominion Centre",0,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Bar,Gastropub,Seafood Restaurant,Italian Restaurant,Deli / Bodega
8,Stn A PO Boxes 25 The Esplanade,0,Coffee Shop,Café,Restaurant,Seafood Restaurant,Italian Restaurant,Beer Bar,Hotel,Farmers Market,Art Gallery,Bakery
9,"First Canadian Place, Underground city",0,Coffee Shop,Café,Steakhouse,Restaurant,Hotel,Bar,Seafood Restaurant,American Restaurant,Asian Restaurant,Deli / Bodega
11,"Richmond, Adelaide, King",0,Coffee Shop,Café,Steakhouse,Bar,Sushi Restaurant,Bakery,Restaurant,Thai Restaurant,American Restaurant,Asian Restaurant
12,"Garden District, Ryerson",0,Clothing Store,Coffee Shop,Cosmetics Shop,Bakery,Café,Fast Food Restaurant,Ramen Restaurant,Restaurant,Bookstore,Bubble Tea Shop


In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Christie,1,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Diner,Convenience Store,Nightclub,Candy Store,Restaurant


In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Rosedale,2,Park,Playground,Trail,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner


In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,"South Niagara, Railway Lands, Island airport, ...",3,Airport Service,Airport Terminal,Airport Lounge,Boutique,Plane,Coffee Shop,Harbor / Marina,Boat or Ferry,Airport Gate,Airport Food Court


In [61]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,"Harbord, University of Toronto",4,Café,Restaurant,Bakery,Bar,Bookstore,Japanese Restaurant,Sandwich Place,Italian Restaurant,Flower Shop,Pub


In this exercise i decided to focus just on the neighborhoods that are members of the "downtown toronto" borough. I follow the example from the lab.

Observations:

Cluster 0: This cluster contains mostly coffee shops and resturants as its most popular venues. Seems like a fun place to live!

Cluster 1: This cluster contains mainly stores and parks/recreation. A more realistic living experience.

Cluster 2: This cluster contains mainly outdoor/active venues. A great place to live if you enjoy an active lifestyle.

Cluster 3: This cluster contains mainly transportation venues. If you travel frequently, this would be a great place to live

Cluster 4: This cluster is the only one with bookstores that come toward the top of the list. It makes sense, since this is named after the university, and would be a good place to live as a student.